## Task 1

Below code computes the RFM Table. Inline comments to help you navigate/

In [1]:
import pandas as pd
df = pd.read_csv('./data/retail.csv')

In [2]:
## Cleaning price column
df['Price']=df['Price'].map(lambda x: x.replace(",",".")).astype("float")

In [3]:
## Compputing revenue
df['Rev'] = df['Price']*df['Quantity']

### RFM Analysis ###
- Find out how many times each customer has bought
- Find out when was the last time each customer bought
- Find out the total and average spend so far

In [4]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
## This is similar to dense_rank() over() in sql
df['Custdate_rank']=df[['Customer ID','InvoiceDate']].\
                                                groupby('Customer ID').\
                                                InvoiceDate.\
                                                transform('rank',method='dense',ascending=False)
df['Today'] = pd.to_datetime("2012-01-01")
df['Duration']=df['Today']-df['InvoiceDate']
## Selecting the most recent transaction for each customer
recency = df[df['Custdate_rank']==1]\
                                    .sort_values('Customer ID',ascending=False)[['Customer ID','Duration']].\
                                    drop_duplicates()
recency = recency.reset_index().\
                        drop('index',axis=1).\
                        sort_values('Customer ID',ascending=False)

In [5]:
### Frequency
freq = df.groupby('Customer ID').size().\
                            reset_index().\
                            sort_values('Customer ID',ascending=False).\
                            rename(columns={0:'Freq'})

In [6]:
### Monetory
monetary = df.groupby('Customer ID').\
                    agg({'Rev':['sum','mean']}).\
                    reset_index()
monetary.columns=['Customer ID','Tot_Rev','Avg_Rev']
monetary = monetary.\
                    sort_values('Customer ID',ascending=False)

In [7]:
rf = pd.merge(recency,freq,on="Customer ID",how="inner")
rfm = pd.merge(rf,monetary,on="Customer ID",how="inner")
rfm.head()

,Customer ID,Duration,Freq,Tot_Rev,Avg_Rev
0,18287.0,21 days 13:37:00,156,4177.89,26.781346
1,18286.0,498 days 12:03:00,70,1188.43,16.977571
2,18285.0,682 days 13:36:00,12,427.00,35.583333
3,18284.0,569 days 11:29:00,29,436.68,15.057931
4,18283.0,31 days 11:01:00,936,2528.65,2.701549


## Task 2

In [8]:
### Pareto for Tot Rev, Avg Rev. Computing the deciles for each relevant column
rfm['Tot_Rev_Deciles'] = pd.qcut(rfm['Tot_Rev'],10)
rfm['Avg_Rev_Deciles'] = pd.qcut(rfm['Avg_Rev'],10)
rfm['Freq_Deciles'] = pd.qcut(rfm['Freq'],10)
rfm['Recency_Deciles'] = pd.qcut(rfm['Duration'],10)

In [9]:
rec_pareto = rfm.groupby('Recency_Deciles')['Tot_Rev'].\
                                        sum().\
                                        reset_index()
rec_pareto['CumSum_Rev'] = rec_pareto['Tot_Rev'].cumsum()
rec_pareto['Total_Rev_Across_Deciles'] = rec_pareto.Tot_Rev.sum()
rec_pareto['Perc_tot_rev'] = rec_pareto['CumSum_Rev']/rec_pareto['Total_Rev_Across_Deciles']

In [10]:
rec_pareto

,Recency_Deciles,Tot_Rev,CumSum_Rev,Total_Rev_Across_Deciles,Perc_tot_rev
0,"(21 days 06:40:59.999999999, 32 days 12:04:36]",5711857.734,5.711858e+06,1.640410e+07,0.348197
1,"(32 days 12:04:36, 43 days 13:59:12]",2968252.921,8.680111e+06,1.640410e+07,0.529143
2,"(43 days 13:59:12, 61 days 10:34:24.000000001]",2275605.871,1.095572e+07,1.640410e+07,0.667865
3,"(61 days 10:34:24.000000001, 87 days 18:32:36....",1533952.931,1.248967e+07,1.640410e+07,0.761375
4,"(87 days 18:32:36.000000017, 135 days 08:26:00]",1262303.344,1.375197e+07,1.640410e+07,0.838325
5,"(135 days 08:26:00, 216 days 09:30:36.000000004]",965468.294,1.471744e+07,1.640410e+07,0.897181
6,"(216 days 09:30:36.000000004, 345 days 09:39:30]",586407.400,1.530385e+07,1.640410e+07,0.932928
7,"(345 days 09:39:30, 436 days 12:19:24]",536943.362,1.584079e+07,1.640410e+07,0.965661
8,"(436 days 12:19:24, 557 days 12:04:00]",398391.840,1.623918e+07,1.640410e+07,0.989947
9,"(557 days 12:04:00, 1083 days 14:05:00]",164913.601,1.640410e+07,1.640410e+07,1.000000


One can see that for recency range of upto 61 to 87 days, around 76% of total revenue is realised. This means that customers with recency of 21 to 87 days contribute 76% of total revenue

## Task 3

In [11]:
rfm['Tot_Rev_Decile_Num'] = pd.qcut(rfm['Tot_Rev'],10,labels=False)
rfm['Avg_Rev_Decile_Num'] = pd.qcut(rfm['Avg_Rev'],10,labels=False)
rfm['Freq_Decile_Num'] = pd.qcut(rfm['Freq'],10,labels=False)
rfm['Recency_Decile_Num'] = pd.qcut(rfm['Duration'],10,labels=False)

In [12]:
recent_freq_dist = rfm[rfm['Recency_Decile_Num']<=4].groupby('Freq_Deciles')['Tot_Rev'].\
                                                        sum().\
                                                        reset_index()

recent_freq_dist['CumSum']=recent_freq_dist['Tot_Rev'].cumsum()
recent_freq_dist['Total']=recent_freq_dist['Tot_Rev'].sum()
recent_freq_dist['Perc_Total']=recent_freq_dist['CumSum']/recent_freq_dist['Total']

In [13]:
recent_freq_dist

,Freq_Deciles,Tot_Rev,CumSum,Total,Perc_Total
0,"(0.999, 8.0]",56005.810,5.600581e+04,1.375197e+07,0.004073
1,"(8.0, 16.0]",90268.210,1.462740e+05,1.375197e+07,0.010637
2,"(16.0, 25.0]",116612.320,2.628863e+05,1.375197e+07,0.019116
3,"(25.0, 37.0]",178436.851,4.413232e+05,1.375197e+07,0.032092
4,"(37.0, 53.0]",295281.692,7.366049e+05,1.375197e+07,0.053564
5,"(53.0, 76.0]",457906.370,1.194511e+06,1.375197e+07,0.086861
6,"(76.0, 114.0]",695721.150,1.890232e+06,1.375197e+07,0.137452
7,"(114.0, 180.0]",1188163.251,3.078396e+06,1.375197e+07,0.223851
8,"(180.0, 319.7]",2408881.321,5.487277e+06,1.375197e+07,0.399017
9,"(319.7, 12780.0]",8264695.826,1.375197e+07,1.375197e+07,1.000000
